In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from google import genai
from google.colab import userdata

# ==============================
# CONFIGURACIÓN DE LA API
# ==============================
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    print("⚠️ No se ha encontrado la clave API. Configúrala antes de ejecutar el programa.")
    exit()

client = genai.Client(api_key=GOOGLE_API_KEY)

# ==============================
# FUNCIÓN DE SCRAPING RECURSIVO EN TODA LA WEB
# ==============================
def obtener_info_web(url, visitadas=set()):
    if url in visitadas:
        return ""

    try:
        respuesta = requests.get(url, timeout=10)
        respuesta.raise_for_status()
        soup = BeautifulSoup(respuesta.text, "html.parser")
        visitadas.add(url)

        # Extraer información de la página actual
        titulo = soup.title.string if soup.title else "Sin título"
        parrafos = [p.text for p in soup.find_all("p")]
        contenido = f"Título: {titulo}\nContenido: {' '.join(parrafos[:50])}"

        # Buscar enlaces internos y hacer scraping en ellos
        enlaces = [urljoin(url, a['href']) for a in soup.find_all('a', href=True) if urljoin(url, a['href']).startswith("https://ilecina.inscastellbisbal.net")]
        for enlace in enlaces:
            contenido += "\n" + obtener_info_web(enlace, visitadas)

        return contenido
    except Exception as e:
        return f"Error al obtener datos de la web: {e}"

# ==============================
# INSTRUCCIONES DEL SISTEMA
# ==============================
system_instruction = """
Eres un asistente de inteligencia artificial que usa información en tiempo real de la web.
Proporciona respuestas claras y basadas en los datos extraídos,omite todo sobre los gustos.
"""

chat = client.chats.create(
    model="gemini-2.0-flash",
    config=genai.types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.7,
        max_output_tokens=200
    )
)

# ==============================
# BUCLE DEL CHAT
# ==============================
print("\n💬 XatBot activado! Escribe 'salir' para terminar la conversación.\n")

while True:
    try:
        prompt = input("👤 Usuario: ").strip()
        if prompt.lower() == "salir":
            print("👋 XatBot: Hasta luego!")
            break

        # Obtener información de toda la web
        info_web = obtener_info_web("https://ilecina.inscastellbisbal.net")

        # Enviar mensaje al modelo con la información extraída
        respuesta = chat.send_message(f"{info_web}\n\nPregunta: {prompt}")

        print(f"🤖 XatBot: {respuesta.text.strip()}\n")
    except Exception as e:
        print(f"⚠️ Error en la comunicación con Gemini: {e}")



💬 XatBot activado! Escribe 'salir' para terminar la conversación.



KeyboardInterrupt: Interrupted by user

In [39]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import google.generativeai as genai
from flask_cors import CORS
from google.colab import userdata

# ==============================
# CONFIGURACIÓN DE LA API
# ==============================
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    print("⚠️ No se ha encontrado la clave API. Configúrala antes de ejecutar el programa.")
genai.configure(api_key=GOOGLE_API_KEY)

# ==============================
# CONFIGURACIÓN DE FLASK
# ==============================
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# ==============================
# CONFIGURACIÓN DE NGROK
# ==============================
NGROK_AUTHTOKEN = userdata.get("NGROK")
ngrok.set_auth_token(NGROK_AUTHTOKEN)
public_url = ngrok.connect(5001)
print(f" * Servidor disponible en: {public_url}")

# ==============================
# FUNCIÓN DE SCRAPING RECURSIVO EN TODA LA WEB
# ==============================
def obtener_info_web(url, visitadas=set()):
    print(f"Scraping página: {url}")
    if url in visitadas:
        return ""

    try:
        respuesta = requests.get(url, timeout=10)
        respuesta.raise_for_status()
        soup = BeautifulSoup(respuesta.text, "html.parser")
        visitadas.add(url)

        # Extraer información de la página actual
        titulo = soup.title.string if soup.title else "Sin título"
        parrafos = [p.text for p in soup.find_all("p")]
        contenido = f"Título: {titulo}\nContenido: {' '.join(parrafos[:50])}"

        # Buscar enlaces internos y hacer scraping en ellos
        enlaces = [urljoin(url, a['href']) for a in soup.find_all('a', href=True) if urljoin(url, a['href']).startswith("https://ilecina.inscastellbisbal.net")]
        for enlace in enlaces:
            contenido += "\n" + obtener_info_web(enlace, visitadas)

        return contenido

    except Exception as e:
        return f"Error al obtener datos de la web: {e}"
        print('error')

# ==============================
# INSTRUCCIONES DEL SISTEMA
# ==============================
system_instruction = """
Eres un asistente de inteligencia artificial que usa información en tiempo real de la web.
Proporciona respuestas claras y basadas en los datos extraídos, omite todo sobre los gustos.
"""

chat = genai.GenerativeModel("gemini-1.5-flash").start_chat()

@app.route('/*', methods=['POST'])
def obtener_respuesta():
    print('trace1')
    if request.method == "OPTIONS":
        return jsonify({"status": "ok"}), 200

    try:
        data = request.get_json()
        if not data or 'mensaje' not in data:
            return jsonify({'error': 'Solicitud inválida, falta "mensaje"'}), 400

        mensaje = data['mensaje']
        # Imprimir el mensaje recibido en los logs de Flask
        print(f"Mensaje recibido: {mensaje}")
        respuesta = f"Respuesta desde el servidor para el mensaje: {mensaje}"

        return jsonify({'respuesta': respuesta}), 200
    except Exception as e:
        # Capturar cualquier error y mostrarlo en los logs
        print(f"Error al procesar la solicitud: {e}")
        return jsonify({'error': 'Error interno en el servidor'}), 500

if __name__ == '__main__':
    app.run(port=5001)


KeyboardInterrupt: 

In [44]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import google.generativeai as genai
from flask_cors import CORS
from google.colab import userdata
!kill -9 $(lsof -t -i:5001)
# ==============================
# CONFIGURACIÓN DE LA API
# ==============================
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    print("⚠️ No se ha encontrado la clave API. Configúrala antes de ejecutar el programa.")
genai.configure(api_key=GOOGLE_API_KEY)

# ==============================
# CONFIGURACIÓN DE FLASK
# ==============================
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# ==============================
# CONFIGURACIÓN DE NGROK
# ==============================
NGROK_AUTHTOKEN = userdata.get("NGROK")
ngrok.set_auth_token(NGROK_AUTHTOKEN)
public_url = ngrok.connect(5001)
print(f" * Servidor disponible en: {public_url}")

# ==============================
# FUNCIÓN DE SCRAPING RECURSIVO EN TODA LA WEB
# ==============================
def obtener_info_web(url, visitadas=set()):
    print(f"Scraping página: {url}")
    if url in visitadas:
        return ""

    try:
        respuesta = requests.get(url, timeout=10)
        respuesta.raise_for_status()
        soup = BeautifulSoup(respuesta.text, "html.parser")
        visitadas.add(url)

        # Extraer información de la página actual
        titulo = soup.title.string if soup.title else "Sin título"
        parrafos = [p.text for p in soup.find_all("p")]
        contenido = f"Título: {titulo}\nContenido: {' '.join(parrafos[:50])}"

        # Buscar enlaces internos y hacer scraping en ellos
        enlaces = [urljoin(url, a['href']) for a in soup.find_all('a', href=True) if urljoin(url, a['href']).startswith("https://ilecina.inscastellbisbal.net")]
        for enlace in enlaces:
            contenido += "\n" + obtener_info_web(enlace, visitadas)

        return contenido
        print('scraping finalizado')
    except Exception as e:
        return f"Error al obtener datos de la web: {e}"

# ==============================
# INSTRUCCIONES DEL SISTEMA
# ==============================
system_instruction = """
Ets un assistent d'intel·ligència artificial que utilitza informació de la web ilecina.inscastellbisbal.net,
Proporciona respostes clares i basades únicament en les dades proporcionades, omet tot sobre els gustos i no mencionis molt el repte1, sóc izan.
"""

# Crear el chat
chat = genai.GenerativeModel("gemini-1.5-flash").start_chat()

@app.route('/', methods=['POST'])
def obtener_respuesta():
    if request.method == "OPTIONS":
        return jsonify({"status": "ok"}), 200

    try:
        data = request.get_json()
        if not data or 'mensaje' not in data:
            return jsonify({'error': 'Solicitud inválida, falta "mensaje"'}), 400

        mensaje = data['mensaje']
        # Imprimir el mensaje recibido en los logs de Flask
        print(f"Mensaje recibido: {mensaje}")

        # Obtener información de toda la web
        info_web = obtener_info_web("https://ilecina.inscastellbisbal.net")

        # Enviar mensaje al modelo con la información extraída
        respuesta = chat.send_message(f"{info_web}\n\nPregunta: {mensaje}")

        print(f"Respuesta del modelo: {respuesta.text.strip()}")

        return jsonify({'respuesta': respuesta.text.strip()}), 200
        print('funciona')

    except Exception as e:
        # Capturar cualquier error y mostrarlo en los logs
        print(f"Error al procesar la solicitud: {e}")
        return jsonify({'error': 'Error interno en el servidor'}), 500

obtener_info_web('https://ilecina.inscastellbisbal.net')
if __name__ == '__main__':
    app.run(port=5001)


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
 * Servidor disponible en: NgrokTunnel: "https://3272-35-236-147-149.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 10:29:11] "OPTIONS / HTTP/1.1" 200 -


Se han truncado las últimas 5000 líneas del flujo de salida.
Scraping página: https://ilecina.inscastellbisbal.net/category/1r/
Scraping página: https://ilecina.inscastellbisbal.net/category/1r/repte-3/
Scraping página: https://ilecina.inscastellbisbal.net/2024/12/14/arrencada-i-parada-de-lequip/
Scraping página: https://ilecina.inscastellbisbal.net/2024/12/14/configuracio-de-chocolatey/
scraping finalizado
Scraping página: https://ilecina.inscastellbisbal.net/2024/12/14/configuracio-de-chocolatey/
Scraping página: https://ilecina.inscastellbisbal.net/repte3/xarxa-sense-fils/
Scraping página: https://ilecina.inscastellbisbal.net/2024/12/15/preparacio-del-ap/
Scraping página: https://ilecina.inscastellbisbal.net/repte3/helpdesk/
Scraping página: https://ilecina.inscastellbisbal.net/problema-wifi/
Scraping página: https://ilecina.inscastellbisbal.net/problema-audio/
Scraping página: https://ilecina.inscastellbisbal.net/problema-video/
Scraping página: https://ilecina.inscastellbisbal.net

INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 10:39:17] "POST / HTTP/1.1" 200 -


Respuesta del modelo: Hola!


...

Respuesta del modelo: Basándome en la información que me has proporcionado previamente, Izan es un estudiante apasionado por la tecnología, los videojuegos y el arte que está cursando un ciclo formativo de grado medio de Sistemas Microinformáticos y Redes en el Instituto Castellbisbal.  Su sueño es convertirse en actor de doblaje.  Ha creado una página web (ilecina.inscastellbisbal.net) donde documenta sus proyectos y actividades del ciclo formativo.


INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 10:41:08] "OPTIONS / HTTP/1.1" 200 -


Mensaje recibido: explicame mas de la web
Scraping página: https://ilecina.inscastellbisbal.net


INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 10:41:17] "POST / HTTP/1.1" 200 -


Respuesta del modelo: La web de Izan,  `ilecina.inscastellbisbal.net`, parece ser un portfolio digital donde documenta su progreso en el ciclo formativo de Sistemas Microinformáticos y Redes.  Basándome en los títulos y contenidos que me proporcionaste, la web incluye información organizada en posts o entradas que detallan diferentes aspectos de sus proyectos:

* **Introducción:** Presenta a Izan, sus intereses y sus objetivos.
* **Retos del ciclo formativo:** La web se estructura en torno a diferentes retos o proyectos que Izan realiza a lo largo de los dos cursos académicos (2024-2025 y 2025-2026).  Cada reto tiene su propia sección con una descripción de las tareas realizadas.
* **Aspectos técnicos:** Se describen tareas relacionadas con la configuración de correo electrónico (corporativo y profesional), la gestión de agendas electrónicas, el uso de plataformas web como NetAcad y Filezilla, la configuración de entornos de trabajo (en la nube y en su ordenador personal),  la instalac

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import google.generativeai as genai
from flask_cors import CORS
from google.colab import userdata
import time

# ==============================
# CONFIGURACIÓ DE LA API
# ==============================
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    print("⚠️ No s'ha trobat la clau API. Configura-la abans d'executar el programa.")
genai.configure(api_key=GOOGLE_API_KEY)

# ==============================
# CONFIGURACIÓ DE FLASK
# ==============================
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# ==============================
# CONFIGURACIÓ DE NGROK
# ==============================
NGROK_AUTHTOKEN = userdata.get("NGROK")
ngrok.set_auth_token(NGROK_AUTHTOKEN)
public_url = ngrok.connect(5001)
print(f" * Servidor disponible a: {public_url}")

# ==============================
# FUNCIÓ DE SCRAPING RECURSIVA
# ==============================
def obtener_info_web(url, visitadas=set(), profundidad=0, max_profundidad=2):
    print(f"Scraping página: {url}")

    # Evitar el scraping de páginas ya visitadas o si se alcanza la profundidad máxima
    if url in visitadas or profundidad >= max_profundidad:
        return ""

    try:
        # Realizar la solicitud HTTP
        respuesta = requests.get(url, timeout=10)
        respuesta.raise_for_status()  # Verificar si hay errores en la respuesta
        soup = BeautifulSoup(respuesta.text, "html.parser")
        visitadas.add(url)  # Marcar la URL como visitada

        # Extraer información de la página
        titulo = soup.title.string if soup.title else "Sin título"
        parrafos = [p.text for p in soup.find_all("p")]
        contenido = f"Título: {titulo}\nContenido: {' '.join(parrafos[:50])}"

        # Esperar antes de hacer nuevas solicitudes para evitar sobrecargar el servidor
        time.sleep(1)

        # Buscar enlaces internos y hacer scraping en ellos (evitar enlaces externos y duplicados)
        enlaces = [urljoin(url, a['href']) for a in soup.find_all('a', href=True)
                   if urljoin(url, a['href']).startswith("https://ilecina.inscastellbisbal.net")]

        for enlace in enlaces:
            if enlace not in visitadas:  # Solo visitar enlaces que no hayan sido procesados
                contenido += "\n" + obtener_info_web(enlace, visitadas, profundidad + 1, max_profundidad)

        return contenido

    except requests.exceptions.RequestException as e:
        return f"Error al obtener datos de la web: {e}"

# Ejemplo de ejecución
# Inicializar el scraping desde la URL principal
contenido_completo = obtener_info_web("https://ilecina.inscastellbisbal.net")
print(contenido_completo)

# ==============================
# INSTRUCCIONS DEL SISTEMA
# ==============================
instruccions_systema = """
Ets un assistent d'intel·ligència artificial que utilitza exclusivament la informació extraïda de la web ilecina.inscastellbisbal.net.
Només pots respondre amb les dades obtingudes mitjançant el scraping de la pàgina web. No utilitzis cap coneixement preexistent o extern, només la informació proporcionada pel scraping.
No facis suposicions ni utilitzis informació que no prové de la web.
Proporciona respostes clares basades únicament en les dades obtingudes.
Omet tota referència a gusts, opinions, i no facis menció del repte1, sóc izan.
"""

# Crear el chat
chat = genai.GenerativeModel("gemini-1.5-flash").start_chat()

@app.route('/', methods=['POST'])
def obtenir_resposta():
    if request.method == "OPTIONS":
        return jsonify({"status": "ok"}), 200

    try:
        dades = request.get_json()
        if not dades or 'mensaje' not in dades:
            return jsonify({'error': 'Solicitud invàlida, falta "mensaje"'}), 400

        mensaje = dades['mensaje']
        # Imprimir el mensaje rebut als logs de Flask
        print(f"missatge rebut: {mensaje}")

        # Obtenir informació de tota la web
        info_web = obtenir_info_web("https://ilecina.inscastellbisbal.net")

        # Enviar mensaje al model amb la informació extraïda
        resposta = chat.send_message(f"{info_web}\n\nPregunta: {mensaje}")

        print(f"Resposta del model: {resposta.text.strip()}")

        return jsonify({'resposta': resposta.text.strip()}), 200
    except Exception as e:
        # Capturar qualsevol error i mostrar-lo als logs
        print(f"Error en processar la sol·licitud: {e}")
        return jsonify({'error': 'Error intern al servidor'}), 500

obtenir_info_web('https://ilecina.inscastellbisbal.net')

if __name__ == '__main__':
    app.run(port=5001)


 * Servidor disponible a: NgrokTunnel: "https://f871-35-236-147-149.ngrok-free.app" -> "http://localhost:5001"
Scraping página: https://ilecina.inscastellbisbal.net
Scraping página: https://ilecina.inscastellbisbal.net#content
Scraping página: https://ilecina.inscastellbisbal.net/ipop/
Scraping página: https://ilecina.inscastellbisbal.net/ipop/#content
Scraping página: https://ilecina.inscastellbisbal.net/privacy-policy-2/
Scraping página: https://ilecina.inscastellbisbal.net/repte1eines-de-treball/
Scraping página: https://ilecina.inscastellbisbal.net/2024/09/20/correu-corporatiu/
Scraping página: https://ilecina.inscastellbisbal.net/2024/09/23/correu-professional/
Scraping página: https://ilecina.inscastellbisbal.net/2024/09/23/agenda-electronica/
Scraping página: https://ilecina.inscastellbisbal.net/2024/09/23/espai-de-treball-al-nuvol/
Scraping página: https://ilecina.inscastellbisbal.net/2024/09/30/espai-de-treball-a-casa/
Scraping página: https://ilecina.inscastellbisbal.net/2024

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 11:47:26] "OPTIONS / HTTP/1.1" 200 -


missatge rebut: hola?
Scraping pàgina: https://ilecina.inscastellbisbal.net


INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 11:47:27] "POST / HTTP/1.1" 200 -


Resposta del model: Hola! ¿Cómo estás?


...

Resposta del model: Sóc un model lingüístic gran, entrenat per Google.  Sé moltes coses!  He estat entrenat amb una enorme quantitat de text i codi, i puc generar text, traduir llengües, escriure diferents tipus de creatiu contingut i respondre a les teves preguntes de la manera més informativa possible.  

Però, la meva capacitat de "saber" està limitada per la informació amb què he estat entrenat.  No tinc experiències personals, ni sentiments, ni consciència.

Per dir-ho d'una altra manera: sé molt sobre moltes coses, però no "sé" res en el sentit humà del terme.  Per poder dir-te el que sé, necessito que em facis una pregunta més específica.  Què vols saber?


INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 11:53:18] "OPTIONS / HTTP/1.1" 200 -


missatge rebut: que es un bitcoin
Scraping pàgina: https://ilecina.inscastellbisbal.net


INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 11:53:22] "POST / HTTP/1.1" 200 -


Resposta del model: Un Bitcoin és una criptomoneda, una moneda digital descentralitzada. Això significa que no està controlada per cap govern o institució financera centralitzada, sinó que funciona a través d'una xarxa descentralitzada peer-to-peer (P2P) que utilitza la tecnologia blockchain.

Aquí hi ha alguns punts clau per entendre què és un Bitcoin:

* **Descentralitzat:**  A diferència de les monedes fiduciàries (com el dòlar o l'euro), el Bitcoin no està emès ni controlat per un banc central o govern.  Això fa que sigui teòricament resistent a la censura i la inflació generada pel govern.

* **Blockchain:** El Bitcoin utilitza una tecnologia de registre distribuït anomenada blockchain. Aquesta és una base de dades compartida públicament i distribuïda entre molts ordinadors (nodes) arreu del món.  Cada transacció es registra en un bloc, i aquests blocs s'enllacen entre ells formant una cadena (blockchain). Això garanteix la transparència i la seguretat de les transaccions.

* **Mi